<a href="https://colab.research.google.com/github/ajay-sainy/notebooks/blob/main/Speaker_diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for speechbrain
!pip install -qq torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -qq speechbrain==0.5.12

# pyannote.audio
!pip install -qq pyannote.audio

# for visualization purposes
!pip install -qq moviepy ipython==7.34.0

# for download
!pip install --upgrade yt-dlp
!pip install --upgrade ffmpeg

     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 49.0 MB/s 
     |████████████████████████████████| 10.4 MB 18.2 MB/s 
     |████████████████████████████████| 496 kB 24.2 MB/s 
     |████████████████████████████████| 120 kB 71.1 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 109 kB 69.9 MB/s 
     |████████████████████████████████| 546 kB 66.1 MB/s 
     |████████████████████████████████| 385 kB 13.5 MB/s 
     |████████████████████████████████| 51 kB 162 kB/s 
     |████████████████████████████████| 419 kB 46.6 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 129 kB 64.3 MB/s 
     |████████████████████████████████| 585 kB 70.9 MB/s 
     |████████████████████████████████| 60 kB 6.2 MB/s 
     |███████████████

In [2]:
youtubeUrl="https://www.youtube.com/watch?v=Gl-9GIvnJ2k"
wavFile="rawInput.mp3"
inputAudio="mono.wav"

!yt-dlp -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 $youtubeUrl -o $wavFile

!ffmpeg -y -i $wavFile "temp.wav"
!ffmpeg -y -i temp.wav -ac 1 $inputAudio

[youtube] Gl-9GIvnJ2k: Downloading webpage
[youtube] Gl-9GIvnJ2k: Downloading android player API JSON
[info] Gl-9GIvnJ2k: Downloading 1 format(s): 251
[download] Destination: rawInput.webm
[download] 100% of 13.49MiB in 00:00 at 33.33MiB/s
[ExtractAudio] Destination: rawInput.mp3
Deleting original file rawInput.webm (pass -k to keep)
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt 

In [3]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization')
diarization = pipeline(inputAudio)

Downloading:   0%|          | 0.00/598 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/318 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [14]:
data = {}
for speaker in diarization.labels():
  data[speaker] = []

for turn, _, speaker in diarization.itertracks(yield_label=True):
  t = (round(turn.start, 2), round(turn.end), 2)
  data[speaker].append(t)
  
print(data)

[(2.94, 5, 2), (9.15, 21, 2), (21.3, 21, 2), (21.39, 21, 2), (21.46, 22, 2), (22.69, 48, 2), (48.78, 52, 2), (52.15, 52, 2), (52.19, 52, 2), (52.79, 60, 2), (60.52, 64, 2), (135.01, 135, 2), (171.73, 189, 2), (191.44, 202, 2), (202.37, 202, 2), (202.76, 203, 2), (244.76, 259, 2), (399.22, 401, 2), (421.8, 424, 2), (425.39, 461, 2), (463.29, 494, 2), (494.97, 501, 2), (502.19, 511, 2), (513.24, 529, 2), (532.04, 537, 2), (537.54, 541, 2), (573.71, 579, 2), (578.67, 581, 2), (822.95, 836, 2), (837.6, 840, 2), (839.96, 840, 2), (840.2, 854, 2), (855.55, 871, 2), (1027.68, 1030, 2)]


In [22]:
!pip install pydub

from pydub import AudioSegment
inputAudioSegment = AudioSegment.from_wav(inputAudio)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
audioDict = {}
for speaker in diarization.labels():
  audioDict[speaker] = AudioSegment.empty()

for speaker in audioDict.keys():
  for interval in data[speaker]:
    segment = inputAudioSegment[interval[0] * 1000 :interval[1] * 1000]
    audioDict[speaker] += segment

  audioDict[speaker].export(f"{speaker}.wav", format="wav")